# <u>27/04/2021  to 02/05/2021 work</u>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

### <u>Data Collection and Preperation</u>
**guys here we can understand how to combine multiple data_sets from a folder kindly go through this guys**

In [ ]:
# files=os.listdir(r'E:\solar_radiation_data')
# files

In [ ]:
# path=r'E:\solar_radiation_data'

# #blank dataframe
# final=pd.DataFrame()

# for file in files:
#     df=pd.read_csv(path+"/"+file,encoding='utf-8')
#     final=pd.concat([df,final])

In [ ]:
final = pd.read_csv("../input/solar-hourly-data/combined_data_from_2016_2019_with_out_extra_features.csv")

In [ ]:
final.head(n=-5)

In [ ]:
final.shape

In [ ]:
final.isnull().sum()

### <u>Correlation Analysis</u>

In [ ]:
final.corr()['GHI']

**since it is observed that "GHi=clearsky Ghi" we can remove clearsky Ghi**
**we can also find that "fill flag","precipitable water" and "pressure" has very Low impact on GHI so there is no 
use of considering those columns**
**<u>so by this correlation analysis I am going to elimanate "clearsky GHI","fill flag","precipitable water" and "pressure" columns from our Data</u>**


In [ ]:
# let's rename our data to df from final to make it easy for analysis 
df = final.drop([ "Clearsky GHI","Fill Flag"], axis = 1)
df.head()

In [ ]:
features = df.columns
for feature in features:
    if (len(df[feature].unique())<50):
        print('for {}:{}'.format(feature,df[feature].unique()))

**from above Observation we can conclude other than date&time columns on cloud type contains no continuous values, rest of all other features contains Continuous features**

## <u>Univariate Analysis</u>

In [ ]:
for feature in features:
    plt.plot(df[feature], '+')
    plt.xlabel(feature)
    plt.title('analysis of'+feature)
    plt.show()

## <u>Bivariate Analysis</u>

In [ ]:
for feature in features:
    df.plot(kind='scatter', x=feature, y='GHI')
    plt.show()

**1. we can observe GHI increases between 6 to 18 hours from hour and GHI plot
  2. we can observe at cloud_type 5,9 there is no impact on GHI
  3. we can also observe increasing Solar_Zenith_angle Decreases GHI
  4. and can say increasing temperature can increase GHI
  
***<u>this is the observation from bivariate analysis between features and "GHI"</u>***

****for <u>"relative Humidity,wind direction,wind speed and dew point"</u> we can't find any relation from this analysis let's try with another****
  

In [ ]:
for feature in features:
    df.plot(kind='scatter', y=feature, x='Month')
    plt.show()

***from above analysis between months and features we canclude that
   1. between march and may GHI was at peak level then its get down fall
   2. wind speed was maximum in month of july and started increasing from April and down fall starts from august***

In [ ]:
for feature in features:
    df.plot(kind='scatter', y=feature, x='Year')
    plt.show()

It is found that surface_albedo is maximum in year 2018

so far analysis can conclude droping DHI, CLearsky DHI,DNI, CLearsky DNI columns

In [ ]:
df.corr()['GHI']

In [ ]:
df.dtypes

In [ ]:
features = df.columns
features

In [ ]:
for i in range(1,len(features)):
    data=df.copy()
#     data[feature]=np.log(data[feature])
    data.boxplot(column=features[i],grid = True)
    plt.ylabel(features[i])
    plt.title(features[i])
    plt.show()

In [ ]:
df.corr()['GHI']

In [ ]:
x = df.drop(['GHI'],axis =1)
y = df['GHI']

## <u>Model Creation</u>

In [ ]:
# # if not installed pycaret
# import sys
# !{sys.executable} -m pip install numpy
# !pip install pycaret

# Auto Pycaret

In [ ]:
import pycaret
from pycaret.regression import *
reg = setup(data=df, target='GHI',remove_outliers=True,feature_selection_threshold=0.2, silent=True, feature_selection=True, transformation=False, train_size=0.8)

In [ ]:
compare_models()

# Train_Test_Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=42)

# Model Fitting

In [ ]:
from sklearn.metrics import accuracy_score

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)
Y_pred_lr = lr.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For Logistic Regression:',round(r2_score(Y_pred_lr,y_test)*100,2))

## K Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train,y_train)
Y_pred_knn=knn.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For K Nearest Neighbours :',round(r2_score(Y_pred_knn,y_test)*100,2))

## xgboost

In [ ]:
# !pip install xgboost

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(verbosity=0) 
xgb_model.fit(x_train, y_train)

Y_pred_xgb = xgb_model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For xgboost Regressor :',round(r2_score(Y_pred_xgb,y_test)*100,2))

In [ ]:
Y_pred_x = xgb_model.predict(x_train)
print('R2 Score For xgboost Regressor :',round(r2_score(Y_pred_x,y_train)*100,2))

In [ ]:
# Feeling that the model was overfitted because testing accuracy was arround 95% and training accuracy was 98% 

## Neural Networks

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu',input_dim=15))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
Y_pred_nn = model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For Neural Networks :',round(r2_score(Y_pred_nn,y_test)*100,2))

In [ ]:
Y_pred_n = model.predict(x_train)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For Neural Networks :',round(r2_score(Y_pred_n,y_train)*100,2))

<b>finally I found Neural Networks was performing well from my conclusion with out any overfitting Problem</b>

# Testing the Model

In [ ]:
x_test[:10]

In [ ]:
k = model.predict(x_test[:10])
k

In [ ]:
y_test[:10]

In [ ]:
df1 = pd.DataFrame(k,columns=['predicted_values'])

In [ ]:
df1

here we can see that predited values are acceptable and model that we created also Generalised model with good training 
and testing accuracies with out overfitting

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, AveragePooling2D

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

In [ ]:
from keras.initializers import RandomNormal
model_relu_ADAM_BN_drop = Sequential()

model_relu_ADAM_BN_drop.add(Dense(128, activation='relu', input_dim=15, kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model_relu_ADAM_BN_drop.add(BatchNormalization())
model_relu_ADAM_BN_drop.add(Dropout(0.3))

model_relu_ADAM_BN_drop.add(Dense(64, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu_ADAM_BN_drop.add(BatchNormalization())
model_relu_ADAM_BN_drop.add(Dropout(0.3))

model_relu_ADAM_BN_drop.add(Dense(64, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.250, seed=None)) )
model_relu_ADAM_BN_drop.add(BatchNormalization())
model_relu_ADAM_BN_drop.add(Dropout(0.3))

model_relu_ADAM_BN_drop.add(Dense(1))

print(model_relu_ADAM_BN_drop.summary())

model_relu_ADAM_BN_drop.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model_relu_ADAM_BN_drop.fit(x_train, y_train, batch_size=128, epochs=10 , verbose=1, validation_data=(x_test, y_test))

In [ ]:
Y_pred_rab = model_relu_ADAM_BN_drop.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For Neural Networks :',round(r2_score(Y_pred_rab,y_test)*100,2))

In [ ]:
Y_pred_r =  model_relu_ADAM_BN_drop.predict(x_train)
Y_pred_rab[:10]

In [ ]:
from sklearn.metrics import r2_score
print('R2 Score For Neural Networks :',round(r2_score(Y_pred_r,y_train)*100,2))

In [ ]:
y_test[:10]

# here we can see that the prediction were some how in accurate so we can't consider this model out of which Neural Networks performed well

## 26/04/2021 work

In [ ]:
# df = pd.read_csv('C:/Users/HP/OneDrive/Desktop/combined_data_from_2016_2019_with_extra_features.csv')

In [ ]:
# df.head(n=200)

In [ ]:
# df.corr()

In [ ]:
# df = df.drop(['Clearsky GHI','DHI','Clearsky DHI','DNI','Clearsky DNI'],axis =1)

In [ ]:
# df.head()

In [ ]:
# print(df.corr()["GHI"].abs().sort_values(ascending=False))

In [ ]:
# df = df.drop(['Ozone','Fill Flag'],axis = 1)

In [ ]:
# print(df.corr()["GHI"].abs().sort_values(ascending=True))

In [ ]:
# df.info()

In [ ]:
# print(np.percentile(df.Dew_Point,[99])[0])
# print(np.percentile(df.Dew_Point,[100])[0])

In [ ]:
# print(np.percentile(df.Solar_Zenith_Angle,[0])[0])
# print(np.percentile(df.Solar_Zenith_Angle,[3])[0])

In [ ]:
# features = df.columns
# features

In [ ]:
# # Linear analysis
# for feature in features:
#     data=df.copy()
#     plt.scatter(data[feature],data['GHI'])
#     plt.xlabel(feature)
#     plt.ylabel('GHI')
#     plt.title(feature)
#     plt.show()
        

In [ ]:
# ## We will be using logarithmic transformation to extract more relavent information from the data


# for feature in features:
#     data=df.copy()
#     data[feature]=np.log(data[feature])
#     data['GHI']=np.log(data['GHI'])
#     plt.scatter(data[feature],data['GHI'])
#     plt.xlabel(feature)
#     plt.ylabel('GHI')
#     plt.title(feature)
#     plt.show()

In [ ]:
# for feature in features:
#     data=df.copy()
#     data[feature]=np.log(data[feature])
#     data.boxplot(column=feature,grid = True)
#     plt.ylabel(feature)
#     plt.title(feature)
#     plt.show()

## found many Outliers from Temp,Dew_poin, Solar_zenith_angle, Surface_albedo Columns

In [ ]:
# from pycaret.regression import *
# reg = setup(data=df, target='GHI',remove_outliers=True, normalize=True, silent=True,train_size=0.8)